# Behavioral analyses FEED project
Replication of Max' analyses.

To do:
* Proper between-subject cross-validation
* Between-subject noise ceiling
* Distance-based features (static and dynamic, i.e., relative to frame 0)
* Other evaluation metric (brier?)
* Rolling count
* kleur modellen (Martinez paper)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os.path as op
from glob import glob

# sklearn stuff
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_predict
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.preprocessing import PolynomialFeatures

# models
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Other
from tqdm import tqdm_notebook
from skimage.transform import rescale
from sklearn.decomposition import PCA
from sklearn.calibration import CalibratedClassifierCV

%matplotlib inline

### Some functions

In [2]:
def _load(sub, target, lags=5):
    
    f = f'../behav_data/preproc/sub-{sub}_task-expressive_ratings.tsv'
    data = pd.read_csv(f, sep='\t', index_col=0)
    
    for rep in [1, 2, 3]:
        for ses in [1, 2, 3]:
            for block in np.arange(1, 15):
                tmp = data.query("rep == @rep & session == @ses & block == @block")
                if tmp.shape[0] == 0:
                    continue
                
                tmp['rating_t_min_1'] = np.r_[np.nan, tmp['rating'].iloc[:-1]]
                data.loc[tmp.index, 'rating_t_min_1'] = tmp['rating_t_min_1']
                for lag in np.arange(2, lags+1):
                    tmp[f'rating_t_min_{lag}'] = np.r_[np.nan, tmp[f'rating_t_min_{lag-1}'].iloc[:-1]]
                    data.loc[tmp.index, f'rating_t_min_{lag}'] = tmp[f'rating_t_min_{lag}']
                    
    data = data.set_index(data.trial_type.astype(str) + '_rep-' + data.rep.astype(str))
    unmelted = pd.pivot(data, columns='rating_type', values='rating')
    data = data.query("rating_type == @target")
    data = pd.concat((data, unmelted), axis=1)
    data = data.sort_values(['rep', 'session', 'run', 'block', 'trial'], axis=0)        
    return data


def get_Xy_from_tsv(sub, feature_space='AU', target='emotion', remove_gva=True, theory_kernel=None,
                    n_comp=50, lags=5):
    
    data = _load(sub, target, lags)
    data = data.query("data_split == 'train'")
    data, test = train_test_split(data, test_size=0.15, random_state=24, stratify=data[target])
    
    # TO FIX: only second argmax from double trials

    if remove_gva:
        data = data.query("emotion != 'Geen van allen'")    
    
    to_return = dict()
    y = data.loc[:, target]
    d2e = dict(Bang='fear', Blij='happy', Boos='angry', Verdrietig='sad', Verrassing='surprise', Walging='disgust')
    to_return['y'] = y.map(d2e)
    to_return['intensity'] = data.loc[:, 'rating_intensity']
    
    if feature_space == 'autocorr':
        cols2use = [f'rating_t_min_{i}' for i in np.arange(1, lags + 1)]
        data = data.loc[:, cols2use]
        for col in cols2use:
            data[col] = data[col].map(d2e)
        
        # only if target == 'emotion'
        ohe = OneHotEncoder(sparse=False)
        cats = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise']
        ohe.fit(np.array(cats)[:, np.newaxis])

        stack = []
        for i, col in enumerate(data.columns):
            tmp = np.zeros((data.shape[0], 6))
            not_nan = ~data[col].isna()
            tmp[not_nan, :] = ohe.transform(data[col].values[not_nan, np.newaxis])
            colnames = [cat + f'_t_min_{i+1}' for cat in cats]
            tmp = pd.DataFrame(tmp, columns=colnames, index=data.index)
            stack.append(tmp)
            
        to_return['X'] = pd.concat(stack, axis=1)
        return to_return
    
    if feature_space in ['pca', 'nmf', 'pcadiff', 'nmfdiff']:
        if feature_space in ['pca', 'nmf']:
            X_red = pd.read_csv(f'../behav_data/features/model-{feature_space}_scale-10_frame-14_features.tsv', sep='\t', index_col=0)
            X_red = X_red.loc[data['trial_type'], :].iloc[:, :n_comp]
            to_return['X'] = X_red.set_index(data.index)         
        else:
            X_red0 = pd.read_csv(f'../behav_data/features/model-{feature_space[:3]}_scale-10_frame-0_features.tsv', sep='\t', index_col=0)
            X_red14 = pd.read_csv(f'../behav_data/features/model-{feature_space[:3]}_scale-10_frame-14_features.tsv', sep='\t', index_col=0)
            X_red0 = X_red0.loc[data['trial_type'], :].iloc[:, :n_comp]
            X_red14 = X_red14.loc[data['trial_type'], :].iloc[:, :n_comp]
            X_red = X_red14 - X_red0
            to_return['X'] = X_red.set_index(data.index)
        return to_return

    if feature_space == 'circumplex':
        to_return['X'] = data.loc[:, ['arousal', 'valence']]
        return to_return
    
    if 'AU' in feature_space or theory_kernel is not None:
        X_AU = data.loc[:, [col for col in data.columns if 'AU' in col]]
        X_AU = X_AU.drop('N_AUs', axis=1)

        if feature_space == 'AU' or theory_kernel is not None:
            to_return['X'] = X_AU
            return to_return
        
        if 'AUxAU' in feature_space:
            poly = PolynomialFeatures(degree=2, interaction_only=True , include_bias=False)
            X_AUxAU = poly.fit_transform(X_AU.values)
            X_AUxAU = pd.DataFrame(X_AUxAU, columns=poly.get_feature_names(X_AU.columns), index=data.index)
        
        if feature_space == 'AUxAU':
            to_return['X'] = X_AUxAU
            return to_return
        
    if 'SA' in feature_space:
        f = f'../behav_data/preproc/sub-{sub}_task-neutral_ratings.tsv'
        data_sa = pd.read_csv(f, sep='\t')
        data_sa = data_sa.set_index(data_sa.trial_type.astype(str) + '_rep-' + data_sa.rep.astype(str))
        unmelted = pd.pivot(data_sa, columns='rating_type', values='rating')
        
        data_sa = pd.concat((data_sa, unmelted), axis=1)
        cols = ['trustworthiness', 'dominance', 'attractiveness', 'valence', 'arousal']
        data_sa = data_sa.loc[:, cols]
        X_SA = data_sa.loc[data.face_id, :]
        X_SA = X_SA.set_index(data.index)
        
        if feature_space == 'SA':
            to_return['X'] = X_SA
            return to_return
        
        elif feature_space == 'AU+SA':
            to_return['X'] = pd.concat((X_AU, X_SA), axis=1)
            return to_return
        
        elif feature_space == 'AUxAU+SA':
            to_return['X'] = pd.concat((X_AUxAU, X_SA), axis=1)
            return to_return

    if feature_space == 'face_attributes':
        X_fa = data.loc[:, ['age', 'gender_F', 'gender_M']]
        to_return['X'] = X_fa
        return to_return
        
    if feature_space == 'nuisance':
        X_nuis = data.loc[:, ['session', 'run', 'block', 'trial', 'onset', 'rating_RT']]
        to_return['X'] = X_nuis
        return to_return
    
    # If we're here, there is a mistake!
    raise ValueError(f"Could not load data for unknown feature_space {feature_space}!")


def brier_score(y_true, y_pred, scale=True, per_class=False):
    
    if y_true.ndim == 1:
        ohe = OneHotEncoder(categories='auto', sparse=False)
        y_true = ohe.fit_transform(y_true[:, np.newaxis])

    if per_class:
        brier = ((y_pred - y_true) ** 2).mean(axis=0)
    else:    
        brier = np.sum((y_pred - y_true) ** 2, axis=1).mean()
    
    if scale:
        if per_class:
            brier_max = np.mean(y_pred, axis=0) * np.mean(1 - y_pred, axis=0)
        else:
            brier_max = np.sum(np.mean(y_pred, axis=0) * np.mean(1 - y_pred, axis=0))
        
        brier = 1 - (brier / brier_max)
        
    return brier


def tjur_score(y_true, y_pred, per_class=False):

    if y_true.ndim == 1:
        ohe = OneHotEncoder(categories='auto', sparse=False)
        y_true = ohe.fit_transform(y_true[:, np.newaxis])

    k = y_true.shape[1]
    score_per_class = np.zeros(k)
    for i in range(k):
        score_0 = y_pred[y_true[:, i] == 0, i].mean()
        score_1 = y_pred[y_true[:, i] == 1, i].mean()
        score_per_class[i] = score_1 - score_0
    
    if per_class:
        return score_per_class
    else:
        return score_per_class.mean()
    

def roc_auc_score_per_class(y_true, y_pred, per_class=True):
    
    if y_true.ndim == 1:
        ohe = OneHotEncoder(categories='auto', sparse=False)
        y_true = ohe.fit_transform(y_true[:, np.newaxis])

    if per_class:
        scores = np.zeros(y_true.shape[1])
        for i in range(y_true.shape[1]):
            scores[i] = roc_auc_score(y_true[:, i], y_pred[:, i])

        return np.array(scores)
    else:
        return roc_auc_score(y_true, y_pred)

def cross_val_predict_and_score(estimator, X, y, cv, scoring, per_class=True, X_cv=None, y_cv=None):
    
    n_class = y.unique().size
    classes = np.sort(y.unique())
    
    if X_cv is not None and y_cv is not None:
        cv_gen = cv.split(X_cv, y_cv)
        preds = pd.DataFrame(np.zeros((y_cv.size, n_class)), columns=classes, index=X_cv.index)
    else:
        cv_gen = cv.split(X, y)
        preds = pd.DataFrame(np.zeros((y.size, n_class)), columns=classes, index=X.index)
    
    estimators = []
    for i, (train_idx_int, test_idx_int) in enumerate(cv_gen):
        
        if X_cv is not None and y_cv is not None:        
            train_idx = X.loc[X_cv.index[train_idx_int], :].dropna().index
            
            test_idx = X.loc[X_cv.index[test_idx_int], :].dropna().index
        else:
            train_idx = X.index[train_idx_int]
            test_idx = X.index[test_idx_int]

        estimator.fit(X.loc[train_idx, :], y.loc[train_idx])
        
        if X_cv is not None and y_cv is not None:
            test_idx = X_cv.index[test_idx_int]
            to_pred = X_cv.loc[test_idx, :]
        else:
            to_pred = X.loc[test_idx, :]
        
        preds.loc[test_idx, :] = estimator.predict_proba(to_pred)        
        estimators.append(estimator)

    y2use = y_cv if y_cv is not None else y
    scores = scoring(y2use.values, preds.values, per_class=per_class)
    
    coef = np.zeros((len(estimators), n_class, X.shape[1]))
    for i, est in enumerate(estimators):
        coef[i, :, :] = est.steps[-1][1].coef_
    
    coef = np.mean(coef, axis=0)
    return scores, coef


scorers = dict(
    brier_score=dict(
        scorer=make_scorer(brier_score, needs_proba=True),
        func=brier_score
    ),
    tjur_score=dict(
        scorer=make_scorer(tjur_score, needs_proba=True),
        func=tjur_score
    ),
    auc_score=dict(
        scorer=make_scorer(roc_auc_score_per_class, needs_proba=True),
        func=roc_auc_score_per_class
    )
)

clfs = dict(
    svm=SVC(kernel='linear', probability=True, class_weight='balanced', decision_function_shape='ovr'),
    lsvm=CalibratedClassifierCV(LinearSVC(class_weight='balanced', multi_class='ovr')),
    lr=LogisticRegression(penalty='elasticnet', class_weight='balanced', solver='saga', l1_ratio=0.5),
    rf=RandomForestClassifier(class_weight='balanced'),
)

gridsearch = dict(
    svm={'C': [0.001, 0.01, 0.1, 1, 10]},
    lr={'C': [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
        'l1_ratio': [0, 0.25, .5, 0.75, 1]},
    rf={'n_estimators': [10, 100, 1000]}    
)

In [3]:
data = get_Xy_from_tsv('01', feature_space='SA')
data['y'].shape

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.



ValueError: Shape of passed values is (500, 72), indices imply (100, 72)

## Noise ceiling

In [ ]:
import warnings
warnings.filterwarnings("ignore")

METRIC = 'auc_score'
TARGET = 'emotion'

files = sorted(glob('../behav_data/preproc/*task-expressive*.tsv'))
cats = ['happy', 'angry', 'fear', 'sad', 'surprise', 'disgust']
le = LabelEncoder()
le.fit(cats)
ohe = OneHotEncoder(categories='auto', sparse=False)
ohe.fit(le.transform(le.classes_)[:, np.newaxis])
print('\t\t', le.classes_)

ceiling = {'sub': [], 'ceiling': [], 'emo': []}
for i, f in enumerate(files):
    
    sub = op.basename(f).split('_')[0].split('-')[1]
    data = _load(sub, TARGET)
    data = data.query("emotion != 'Geen van allen'") 
    data = data.set_index("trial_type")
    
    d2e = dict(Bang='fear', Blij='happy', Boos='angry', Verdrietig='sad', Verrassing='surprise', Walging='disgust')
    data['emotion'] = [d2e[s] for s in data['emotion']]

    dup_idx = data.query("rep == 2").index
    dup_df = pd.DataFrame({
        'rep1': data.query("rep == 1").loc[dup_idx, 'emotion'],
        'rep2': data.query("rep == 2").loc[dup_idx, 'emotion'],
        'rep3': data.query("rep == 3").loc[dup_idx, 'emotion']
    })
    
    dup_df = dup_df.dropna(how='any', axis=0)
    y_flat = data.loc[dup_df.index, 'emotion']

    y = ohe.transform(le.transform(y_flat)[:, np.newaxis]).astype(int)
    y2d = le.transform(dup_df.values.ravel()).reshape(dup_df.shape)
    counts = np.apply_along_axis(np.bincount, 1, y2d, minlength=6)
    best_pred = np.zeros_like(counts, dtype=float)
    for ii in range(counts.shape[0]):
        opt_class = np.where(counts[ii, :] == counts[ii, :].max())[0]
        best_pred[ii, opt_class] = 1 / len(opt_class)

    best_pred_rep = []
    for i in range(best_pred.shape[0]):
        x = best_pred[np.newaxis, i, :]
        best_pred_rep.append(np.r_[x, x, x])
    best_pred_rep = np.vstack(best_pred_rep)
    
    ceil = scorers[METRIC]['func'](y, best_pred_rep, per_class=True)
    print(f"{sub} ceiling: {np.round(ceil, 6)}")
    
    ceiling['sub'].extend([sub] * 6)
    ceiling['ceiling'].extend(ceil.tolist())
    ceiling['emo'].extend(le.classes_)
    
ceiling = pd.DataFrame(ceiling)

## Different feature spaces

In [ ]:
METRIC = 'auc_score'
CLF = 'lr'
N_JOBS = 5
GS = False
SD = True
N_SPLITS = 2
PER_CLASS = True

if GS:
    clf = GridSearchCV(
        estimator=clfs[CLF],
        param_grid=gridsearch[CLF],
        cv=5,
        iid=True,
        n_jobs=N_JOBS,
        scoring=scorers[METRIC]['scorer']
    )
else:
    clf = clfs[CLF]

if SD:
    pipe = make_pipeline(StandardScaler(), clf)
else:
    pipe = clfs[CLF]

cv = StratifiedKFold(n_splits=N_SPLITS)

files = sorted(glob('../behav_data/preproc/*task-expressive*.tsv'))
results = {'sub': [], 'score': [], 'model': [], 'emo': [], 'feature_space': [], 'type': []}
coef_dfs = dict()

# also possible: 'subset_AU'
for feature_space in ['SA', 'circumplex', 'AU', 'AUxAU', 'AU+SA', 'AUxAU+SA', 'nuisance', 'face_attributes']:
    coef_dfs[feature_space] = []
    print('\n%s' % feature_space)
    for i, f in enumerate(files):

        sub = op.basename(f).split('_')[0]
        dat = get_Xy_from_tsv(sub.split('-')[1], feature_space=feature_space)
        X, y, intensity = dat['X'], dat['y'], dat['intensity']
        scores, coef = cross_val_predict_and_score(pipe, X, y, cv=cv, scoring=scorers[METRIC]['func'])
        print(f"{sub}:\t {np.round(scores, 3)}")
        
        if hasattr(scores, '__iter__'):
            mult = scores.size
        else:
            scores = [scores]
            mult = 1

        coef_df = pd.DataFrame(data=coef, columns=X.columns)
        coef_df['sub'] = sub
        coef_df['emo'] = le.classes_[:mult]
        coef_dfs[feature_space].append(coef_df)
        
        results['sub'].extend([sub] * mult)
        results['score'].extend(scores)
        results['model'].extend(['self'] * mult)
        results['emo'].extend(le.classes_[:mult])
        results['feature_space'].extend([feature_space] * mult)
        results['type'].extend(['AU-based'] * mult)
        
    coef_dfs[feature_space] = pd.concat(coef_dfs[feature_space], axis=0)

In [ ]:
results_df = pd.DataFrame(results)
sns.catplot(x='feature_space', y='score', hue='emo', data=results_df,
            kind="bar", aspect=3, height=5, ci='sd')
plt.axhline(0.5, ls='--', c='k')
ax = sns.stripplot(x='feature_space', y='score', hue='emo', edgecolor='black',
                   data=results_df, dodge=True, jitter=False)
ax.legend_.remove()

In [ ]:
sns.catplot(x='emo', y='score', hue='feature_space', data=results_df,
            kind="bar", aspect=3, height=5, ci='sd')
plt.axhline(0.5, ls='--', c='k')
ax = sns.stripplot(x='emo', y='score', hue='feature_space', edgecolor='black',
                   data=results_df, dodge=True, jitter=False)
ax.legend_.remove()

In [ ]:
tmp = pd.melt(coef_dfs['AU'], id_vars=['sub', 'emo'],var_name='feature', value_name='coef')
sns.catplot(x='feature', y='coef', data=tmp, hue='emo',
            kind="bar", aspect=3, height=5, ci='sd')
plt.axhline(0., ls='--', c='k')
ax = sns.stripplot(x='feature', y='coef', hue='emo', edgecolor='black',
                   data=tmp, dodge=True, jitter=False)
ax.legend_.remove()

In [ ]:
tmp = pd.melt(coef_dfs['circumplex'], id_vars=['sub', 'emo'],var_name='feature', value_name='coef')
sns.catplot(x='feature', y='coef', data=tmp, hue='emo',
            kind="bar", aspect=3, height=5, ci='sd')
plt.axhline(0., ls='--', c='k')
ax = sns.stripplot(x='feature', y='coef', hue='emo', edgecolor='black',
                   data=tmp, dodge=True, jitter=False)
ax.legend_.remove()

In [ ]:
tmp = pd.melt(coef_dfs['SA'], id_vars=['sub', 'emo'],var_name='feature', value_name='coef')
sns.catplot(x='feature', y='coef', data=tmp, hue='emo',
            kind="bar", aspect=3, height=5, ci='sd')
plt.axhline(0., ls='--', c='k')
ax = sns.stripplot(x='feature', y='coef', hue='emo', edgecolor='black',
                   data=tmp, dodge=True, jitter=False)
ax.legend_.remove()

In [ ]:
tmp = pd.melt(coef_dfs['nuisance'], id_vars=['sub', 'emo'],var_name='feature', value_name='coef')
sns.catplot(x='feature', y='coef', data=tmp, hue='emo',
            kind="bar", aspect=3, height=5, ci='sd')
plt.axhline(0., ls='--', c='k')
ax = sns.stripplot(x='feature', y='coef', hue='emo', edgecolor='black',
                   data=tmp, dodge=True, jitter=False)
ax.legend_.remove()

In [ ]:
tmp = pd.melt(coef_dfs['face_attributes'], id_vars=['sub', 'emo'],var_name='feature', value_name='coef')
sns.catplot(x='feature', y='coef', data=tmp, hue='emo',
            kind="bar", aspect=3, height=5, ci='sd')
plt.axhline(0., ls='--', c='k')
ax = sns.stripplot(x='feature', y='coef', hue='emo', edgecolor='black',
                   data=tmp, dodge=True, jitter=False)
ax.legend_.remove()

### Decomposition methods

In [ ]:
coef_dfs = dict()
results_dm = {'sub': [], 'score': [], 'model': [], 'emo': [], 'feature_space': [], 'type': []}
for feature_space in ['nmfdiff', 'pca', 'pcadiff', 'nmf']:
    for n_comp in [50, 100, 200]:
        fs_name = feature_space + f':{n_comp}'
        print('\n%s' % fs_name)
        
        coef_dfs[fs_name] = []
        for i, f in enumerate(files):

            sub = op.basename(f).split('_')[0]
            dat = get_Xy_from_tsv(sub.split('-')[1], feature_space=feature_space, n_comp=n_comp)
            X, y, intensity = dat['X'], dat['y'], dat['intensity']
            preds = cross_val_predict(pipe, X, y, cv=cv, n_jobs=1, method='predict_proba')
            scores, coef = cross_val_predict_and_score(pipe, X, y, cv=cv, scoring=scorers[METRIC]['func'])
            print(f"{sub}:\t {np.round(scores, 3)}")

            if hasattr(scores, '__iter__'):
                mult = scores.size
            else:
                scores = [scores]
                mult = 1

            coef_df = pd.DataFrame(data=coef, columns=X.columns)
            coef_df['sub'] = sub
            coef_df['emo'] = le.classes_[:mult]
            coef_dfs[fs_name].append(coef_df)

            results_dm['sub'].extend([sub] * mult)
            results_dm['score'].extend(scores)
            results_dm['model'].extend(['self'] * mult)
            results_dm['emo'].extend(le.classes_[:mult])
            results_dm['feature_space'].extend([fs_name] * mult)
            results_dm['type'].extend(['dimred'] * mult)

        coef_dfs[fs_name] = pd.concat(coef_dfs[fs_name], axis=0)

In [ ]:
results_dm_df = pd.DataFrame(results_dm)
sns.catplot(x='feature_space', y='score', hue='emo', data=results_dm_df,
            kind="bar", aspect=3, height=5, ci='sd')
plt.axhline(0.5, ls='--', c='k')
ax = sns.stripplot(x='feature_space', y='score', hue='emo', edgecolor='black',
                   data=results_dm_df, dodge=True, jitter=False)
ax.legend_.remove()

### Visualize features

In [ ]:
weights_nmf = np.load('../behav_data/features/model-nmf_scale-10_weights.npy')
plt.figure(figsize=(12, 15))
for i in range(25):
    plt.subplot(5, 5, i+1)
    this = weights_nmf[i, :].reshape((80, 60))
    plt.imshow(this)
    plt.axis('off')
    
plt.tight_layout()

In [ ]:
features_per_emo = coef_dfs['nmf:100'].groupby('emo').mean()
fig, ax = plt.subplots(figsize=(20, 15), ncols=6)

for i, emo in enumerate(features_per_emo.index):
    params = features_per_emo.loc[emo, :].values
    these_weights = weights_nmf[:params.size, :]
    viz = params @ these_weights
    ax[i].imshow(viz.reshape((80, 60)))
    ax[i].set_title(emo, fontsize=15)
    ax[i].axis('off')

In [ ]:
weights_pca = np.load('../behav_data/features/model-pca_scale-10_weights.npy')
plt.figure(figsize=(12, 15))
for i in range(25):
    plt.subplot(5, 5, i+1)
    this = weights_pca[i, :].reshape((80, 60))
    plt.imshow(this)
    plt.axis('off')
    
plt.tight_layout()

In [ ]:
features_per_emo = coef_dfs['pca:100'].groupby('emo').mean()
fig, ax = plt.subplots(figsize=(20, 15), ncols=6)

for i, emo in enumerate(features_per_emo.index):
    params = features_per_emo.loc[emo, :].values
    these_weights = weights_pca[:params.size, :]
    viz = params @ these_weights
    ax[i].imshow(viz.reshape((80, 60)))
    ax[i].set_title(emo, fontsize=15)
    ax[i].axis('off')

### Theory kernels

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from scipy.special import softmax
from sklearn.preprocessing import LabelEncoder


class TheoryKernelClassifier(BaseEstimator, ClassifierMixin):  
    """An example of classifier"""

    def __init__(self, au_dict, binarize=False, normalize=True, prob_norm='softmax'):
        
        self.normalize = normalize
        self.binarize = binarize
        self.prob_norm = prob_norm
        self.aus = ['AU1', 'AU10Open', 'AU11', 'AU12', 'AU13', 'AU14', 'AU15',
                    'AU16Open', 'AU17', 'AU2', 'AU20', 'AU22', 'AU24', 'AU25',
                    'AU26', 'AU27i', 'AU4', 'AU43', 'AU5', 'AU6', 'AU7', 'AU9']
        
        self.au_dict = au_dict        
        for emo, cfg in self.au_dict.items():
            if isinstance(cfg, dict):
                vec = np.zeros((len(cfg), len(self.aus)))
                for i, combi in cfg.items():
                    for c in combi:
                        vec[i-1, self.aus.index(c)] = 1
            else:
                vec = np.zeros(len(self.aus))
                for c in cfg:
                    vec[self.aus.index(c)] = 1

            setattr(self, emo, vec)
            
        le = LabelEncoder().fit(['happy', 'sad', 'disgust', 'angry', 'surprise', 'fear'])
        self.le = le
        self.ohe = OneHotEncoder(categories='auto', sparse=False)
        self.ohe.fit(le.transform(le.classes_)[:, np.newaxis])
        self.classes_ = self.le.classes_
        
    def fit(self, X, y=None):
        pass
    
    def predict_proba(self, X, y=None):
        probs = self._predict(X, y)
        return probs  

    def predict(self, X, y=None, string=False):
        probs = self._predict(X, y)
        ties = np.squeeze(probs == probs.max())
        if np.sum(ties) > 0:
            pred = np.random.choice(np.arange(ties.size)[ties])
        else:
            pred = np.argmax(probs)
            
        if string:
            pred = self.le.inverse_transform([pred])

        return pred
        
    def _predict(self, X, y=None):
        
        if X.ndim != 2:
            raise ValueError("X is not 2D.")
        
        if X.shape[1] != 22:
            raise ValueError("This classifier only works with exactly 22 AUs.")
        
        if self.binarize:
            X = (X > 0).astype(int)
            
        self.distances = np.zeros((X.shape[0], 6))
        for key in self.au_dict.keys():
            idx = self.le.transform([key])[0]
            vec = getattr(self, key)
            
            if vec.ndim > 1:
                dist_all = np.zeros((vec.shape[0], X.shape[0]))
                for i in range(vec.shape[0]):
                    sqdist = (X - vec[i, :]) ** 2
                    if self.normalize:
                        sqdist = sqdist / vec[i, :].sum()
                    
                    dist_all[i, :] = np.sqrt(np.sum(sqdist, axis=1))
                dist = np.mean(dist_all,  axis=0)  # or max() ???
            else:
                sqdist = (X - vec) ** 2
                if self.normalize:
                    sqdist = sqdist / vec.sum()
                
                dist = np.sqrt(np.sum(sqdist, axis=1))
            
            self.distances[:, idx] = dist
        
        EPS = 1e-10
        self.distances[self.distances == 0] = EPS
            
        if self.prob_norm == 'softmax':
            sm_dist = softmax(1 / self.distances, axis=1)
        elif self.prob_norm == 'sum':
            sm_dist = (1 / self.distances)
            
            sm_dist = sm_dist / sm_dist.sum(axis=1)[:, np.newaxis]

        return sm_dist

In [ ]:
theory_kernels = dict(
    
    IMotions=dict(
        happy=['AU6', 'AU12'],
        sad=['AU1', 'AU4', 'AU15'],
        surprise=['AU1', 'AU2', 'AU5', 'AU26'],
        fear=['AU1', 'AU2', 'AU4', 'AU5', 'AU7', 'AU20', 'AU26'],
        angry=['AU4', 'AU5', 'AU7'],  # misses AU23
        disgust=['AU9', 'AU15', 'AU16Open']
    ),
    FaceReader=dict(
        happy=['AU6', 'AU12'],
        sad=['AU1', 'AU4'],
        surprise=['AU1', 'AU2'],
        fear=['AU1', 'AU2', 'AU4'],
        angry=['AU4', 'AU5'],
        disgust=['AU10Open', 'AU25']
    ),
    Darwin=dict(
        happy=['AU6', 'AU12'],
        sad=['AU1', 'AU15'],
        surprise={
            1: ['AU1', 'AU2', 'AU5', 'AU25'],
            2: ['AU1', 'AU2', 'AU5', 'AU26']
        },
        fear=['AU1', 'AU2', 'AU5', 'AU20'],
        angry=['AU4', 'AU5', 'AU24'],  # misses AU38
        disgust={
            1: ['AU10Open', 'AU16Open', 'AU22', 'AU25'],
            2: ['AU10Open', 'AU16Open', 'AU22', 'AU26']
        }
    ),
    Matsumoto2008=dict(
        happy=['AU6', 'AU12'],
        sad={
            1: ['AU1', 'AU15'],
            2: ['AU4'],
            3: ['AU4', 'AU1', 'AU15'],
            4: ['AU17'],
            5: ['AU17', 'AU1', 'AU15'],
            6: ['AU17', 'AU1', 'AU15', 'AU4']
        },
        surprise={
            1: ['AU1', 'AU2', 'AU5', 'AU25'],
            2: ['AU1', 'AU2', 'AU5', 'AU26']
        },
        fear={
            1: ['AU1', 'AU2', 'AU4', 'AU5', 'AU20'],
            2: ['AU25'],
            3: ['AU26']
        },
        angry={
            1: ['AU4', 'AU5', 'AU22', 'AU24'],  # misses AU23
            2: ['AU4', 'AU7', 'AU22', 'AU24']   # misses AU23
        },
        disgust={
            1: ['AU9'],
            2: ['AU10Open'],
            3: ['AU25'],
            4: ['AU26'],
            5: ['AU9', 'AU25'],
            6: ['AU10Open', 'AU26']
        }
    ),
    Keltner2019=dict(
        happy=['AU6', 'AU7', 'AU12', 'AU25', 'AU26'],
        sad=['AU1', 'AU4', 'AU6', 'AU15', 'AU17'],
        surprise=['AU1', 'AU2', 'AU5', 'AU25', 'AU26'],
        fear=['AU1', 'AU2', 'AU4', 'AU5', 'AU7', 'AU20', 'AU25'],
        angry=['AU4', 'AU5', 'AU17', 'AU24', ],  # misses AU23 
        disgust=['AU7', 'AU9', 'AU25', 'AU26']  # misses AU19
    ),
    Cordaro2008ref=dict(
        happy=['AU6', 'AU12'],
        sad=['AU1', 'AU4', 'AU5'],
        surprise=['AU1', 'AU2', 'AU5', 'AU26'],
        fear=['AU1', 'AU2', 'AU4', 'AU5', 'AU7', 'AU20', 'AU26'],
        angry=['AU4', 'AU5', 'AU7'],  # misses AU23
        disgust=['AU9', 'AU15', 'AU16Open']
    ),
    Cordaro2008IPC=dict(
        happy={
            1: ['AU6', 'AU7', 'AU12', 'AU16Open', 'AU25', 'AU26'],
            2: ['AU6', 'AU7', 'AU12', 'AU16Open', 'AU25', 'AU27i'],
        },
        sad=['AU4', 'AU43'],  # misses AU54 (head down)
        surprise=['AU1', 'AU2', 'AU5', 'AU25'],
        fear=['AU1', 'AU2', 'AU5', 'AU7', 'AU25'],  # also "jaw"/"move back"
        angry=['AU4', 'AU7'],
        disgust=['AU4', 'AU6', 'AU7', 'AU9', 'AU10Open', 'AU25', 'AU26']  # also "jaw"
    )
)

clf = TheoryKernelClassifier(au_dict=theory_kernels['Darwin'])

In [ ]:
results_tk = {'sub': [], 'score': [], 'model': [], 'emo': [], 'feature_space': [], 'type': []}

for predef_set in ['IMotions', 'FaceReader', 'Darwin', 'Matsumoto2008', 'Keltner2019', 'Cordaro2008ref', 'Cordaro2008IPC']:
    
    clf = TheoryKernelClassifier(
        theory_kernels[predef_set],
        normalize=True,
        binarize=False,
        prob_norm='softmax'
    )
    
    print(f'\n{predef_set}')
    for i, f in enumerate(files):

        sub = op.basename(f).split('_')[0]
        dat = get_Xy_from_tsv(sub.split('-')[1], feature_space='AU',
                              theory_kernel=predef_set)
        X, y, intensity = dat['X'], dat['y'], dat['intensity']
            preds = cross_val_predict(clf, X, y, cv=cv, n_jobs=1, method='predict_proba')
        scores = scorers[METRIC]['func'](y, preds, per_class=PER_CLASS)
        print(f"{sub}:\t {np.round(scores, 3)}")

        if hasattr(scores, '__iter__'):
            mult = scores.size
        else:
            scores = [scores]
            mult = 1

        results_tk['sub'].extend([sub] * mult)
        results_tk['score'].extend(scores)
        results_tk['model'].extend(['self'] * mult)
        results_tk['emo'].extend(le.classes_[:mult])
        results_tk['feature_space'].extend(['TK:%s' % predef_set] * mult)
        results_tk['type'].extend(['theory'] * mult)

In [ ]:
results_tk_df = pd.DataFrame(results_tk)
g = sns.catplot(
    x='feature_space', y='score', hue='emo', data=results_tk_df,
    kind="bar", aspect=3, height=5, ci='sd'
)
plt.axhline(0.5, ls='--', c='k')
ax = sns.stripplot(
    x='feature_space', y='score', hue='emo', edgecolor='black',
    data=results_tk_df, dodge=True, jitter=False
)
ax.legend_.remove()

In [ ]:
results_tk_df = pd.DataFrame(results_tk)
sns.catplot(x='emo', y='score', hue='feature_space', data=results_tk_df,
            kind="bar", aspect=3, height=5, ci='sd')
plt.axhline(0.5, ls='--', c='k')
ax = sns.stripplot(x='emo', y='score', hue='feature_space', edgecolor='black',
              data=results_tk_df, dodge=True, jitter=False)
ax.legend_.remove()

In [ ]:
concat_df = pd.concat((results_df, results_dm_df, results_tk_df), axis=0)
plt.figure(figsize=(15, 5))
sns.catplot(x='feature_space', y='score', data=concat_df, hue='emo',
            kind="bar", aspect=3, height=5, ci='sd')

plt.axhline(0.5, ls='--', c='k')
ax = sns.stripplot(x='feature_space', y='score', hue='emo', edgecolor='black',
                   data=concat_df, dodge=True, jitter=False)
ax.legend_.remove()

## Predict one subject by others

In [ ]:
METRIC = 'auc_score'
CLF = 'lr'
N_JOBS = 5
GS = False
SD = True
N_SPLITS = 10
PER_CLASS = True

if GS:
    clf = GridSearchCV(
        estimator=clfs[CLF],
        param_grid=gridsearch[CLF],
        cv=5,
        iid=True,
        n_jobs=N_JOBS,
        scoring=scorers[METRIC]['scorer']
    )
else:
    clf = clfs[CLF]

if SD:
    pipe = make_pipeline(StandardScaler(), clf)
else:
    pipe = clfs[CLF]

cv = StratifiedKFold(n_splits=N_SPLITS)

files = sorted(glob('../behav_data/preproc/*task-expressive*.tsv'))

results = {'sub': [], 'score': [], 'model': [], 'emo': [], 'feature_space': [], 'type': []}
coef_dfs = dict()

for feature_space in ['SA', 'circumplex', 'AU', 'AUxAU', 'AU+SA', 'AUxAU+SA', 'nuisance', 'face_attributes']:
    coef_dfs[feature_space] = []
    print('\n%s' % feature_space)

    for i, f in enumerate(files):

        sub = op.basename(f).split('_')[0]
        dat = get_Xy_from_tsv(sub.split('-')[1], feature_space=feature_space)
        X, y, intensity = dat['X'], dat['y'], dat['intensity']
        scores, coef = cross_val_predict_and_score(pipe, X, y, cv=cv, scoring=scorers[METRIC]['func'])
        print(f"\n{sub} own model:\t {np.round(scores, 3)}")

        if hasattr(scores, '__iter__'):
            mult = scores.size
        else:
            scores = [scores]
            mult = 1

        coef_df = pd.DataFrame(data=coef, columns=X.columns)
        coef_df['sub'] = sub
        coef_df['emo'] = le.classes_[:mult]
        coef_dfs[feature_space].append(coef_df)
        
        results['sub'].extend([sub] * mult)
        results['score'].extend(scores)
        results['model'].extend(['self'] * mult)
        results['emo'].extend(le.classes_[:mult])
        results['feature_space'].extend([feature_space] * mult)
        results['type'].extend(['AU-based'] * mult)
        
        all_others = [get_Xy_from_tsv(op.basename(fo).split('_')[0].split('-')[1], feature_space=feature_space)
                      for fo in files if fo != f]
        Xall = pd.concat([a['X'] for a in all_others], axis=0)
        yall = pd.concat([a['y'] for a in all_others], axis=0)
        
        scores, coefs = cross_val_predict_and_score(
            pipe, Xall, yall, cv=cv, scoring=scorers[METRIC]['func'],
            X_cv=X, y_cv=y
        )
        print(f"Predicted by all others: {np.round(scores, 3)}")

        coef_df = pd.DataFrame(data=coef, columns=X.columns)
        coef_df['sub'] = sub
        coef_df['emo'] = le.classes_[:mult]
        coef_dfs[feature_space].append(coef_df)
        
        results['sub'].extend([sub] * mult)
        results['score'].extend(scores)
        results['model'].extend(['all_other'] * mult)
        results['emo'].extend(le.classes_[:mult])
        results['feature_space'].extend([feature_space] * mult)
        results['type'].extend(['AU-based'] * mult)
        
        for ii, fo in enumerate(files):
            if f == fo:
                continue

            sub_other = op.basename(fo).split('_')[0]
            dat = get_Xy_from_tsv(sub_other.split('-')[1], feature_space=feature_space)
            Xo, yo = dat['X'], dat['y']
            scores, coefs = cross_val_predict_and_score(
                pipe, Xo, yo, cv=cv, scoring=scorers[METRIC]['func'],
                X_cv=X, y_cv=y
            )
            #print(f"Predicted by {sub_other}:\t {np.round(scores, 3)}")

            coef_df = pd.DataFrame(data=coef, columns=X.columns)
            coef_df['sub'] = sub
            coef_df['emo'] = le.classes_[:mult]
            coef_dfs[feature_space].append(coef_df)

            results['sub'].extend([sub] * mult)
            results['score'].extend(scores)
            results['model'].extend(['other'] * mult)
            results['emo'].extend(le.classes_[:mult])
            results['feature_space'].extend([feature_space] * mult)
            results['type'].extend(['AU-based'] * mult)

    coef_dfs[feature_space] = pd.concat(coef_dfs[feature_space], axis=0)
    
df = pd.DataFrame(results)

### Some plots

In [ ]:
results_df = pd.DataFrame(results)
sns.catplot(x='emo', y='score', hue='model', data=results_df,
            kind="bar", aspect=3, height=5, ci='sd', row='feature_space')
plt.axhline(0.5, ls='--', c='k')
#ax = sns.stripplot(x='feature_space', y='score', hue='emo', edgecolor='black',
#                   data=results_df, dodge=True, jitter=False)
#ax.legend_.remove()

In [ ]:
from matplotlib.lines import Line2D

df = pd.DataFrame(results)

fig, axes = plt.subplots(nrows=4, ncols=3, sharex=True, sharey=True, figsize=(20, 20))
subs = df['sub'].unique()
for i, ax in enumerate(axes.flatten()[:-1]):
    this_sub = subs[i]
    tmp = df.query("sub == @this_sub")
    tmp.loc[tmp['model'].str.contains('sub'), 'model'] = 'other'
    #g = sns.stripplot(x='emo', y='score', hue='model', data=tmp, ax=ax)
    
    sns.stripplot(x='emo', y='score', hue='model', data=tmp.query("model == 'other'"),
                  zorder=1, ax=ax, alpha=0.3, color='black')
    
    sns.pointplot(x='emo', y='score', data=tmp.query("model == 'other'"),
                  markers="x", join=False, ci=None, palette="dark", ax=ax, dodge=0.5)
    
    sns.pointplot(x='emo', y='score', data=tmp.query("model == 'self'"),
                  markers="*", join=False, ci=None, palette="dark", ax=ax, scale=1.5)
    
    sns.pointplot(x='emo', y='score', data=tmp.query("model == 'all_other'"),
                  markers="o", join=False, ci=None, palette="dark", ax=ax)
    
    ax.get_legend().remove()
    ax.set_xlabel('')
    ax.set_title(this_sub, fontsize=15)
    #ax.set_ylim(-0.3, 0.6)

sns.despine()
handles = [
    Line2D([0], [0], marker='x', color='black', label='mean(other)', linestyle='None', markersize=13),
    Line2D([0], [0], marker='*', color='black', label='self', linestyle='None', markersize=13),
    Line2D([0], [0], marker='o', color='black', label='all others', linestyle='None', markersize=13),          
]
axes[0, 1].legend(handles=handles, frameon=False, loc='center', fontsize=15)
fig.tight_layout()

In [ ]:
plt.figure(figsize=(15, 5))
sns.pointplot(x='sub', y='score', hue='model', data=df,
              markers=["x", "o", "*"], scale=1.2, join=False, ci=None, palette="dark")

sns.pointplot(x='sub', y='ceiling', data=ceiling, ci=False, join=False, markers='_', scale=2)

In [ ]:
plt.figure(figsize=(15, 5))
sns.pointplot(x='emo', y='score', hue='model', data=df,
              markers=["x", "o", "*"], scale=1.2, join=False, ci=None, palette="dark")
sns.pointplot(x='emo', y='ceiling', data=ceiling, ci=False, join=False, markers='_', scale=2)

### Distributions of social attribute ratings

In [ ]:
files = sorted(glob('../behav_data/preproc/*task-neutral*.tsv'))

fig, axes = plt.subplots(nrows=len(files), sharex=True, sharey=False, ncols=5, figsize=(15, 3*len(files)))
for i, f in enumerate(files):
    sub = op.basename(f).split('_')[0]
    sdata = pd.read_csv(f, sep='\t')
    
    for ii, attr in enumerate(['valence', 'arousal', 'attractiveness', 'dominance', 'trustworthiness']):
        sns.distplot(np.nan_to_num(sdata[f'rating_{attr}']), ax=axes[i, ii])
        axes[i, ii].set_xlim(-1, 1)
        
sns.despine()
fig.tight_layout()